# Introduction

In this guide, we will walk you through building a powerful semantic search engine using Couchbase as the backend database, [Hugging Face](https://huggingface.co/) as the AI-powered embedding Model. Semantic search goes beyond simple keyword matching by understanding the context and meaning behind the words in a query, making it an essential tool for applications that require intelligent information retrieval. This tutorial is designed to be beginner-friendly, with clear, step-by-step instructions that will equip you with the knowledge to create a fully functional semantic search system from scratch.

# How to run this tutorial

This tutorial is available as a Jupyter Notebook (`.ipynb` file) that you can run interactively. You can access the original notebook [here](https://github.com/couchbase-examples/vector-search-cookbook/blob/main/huggingface/hugging_face.ipynb).

You can either download the notebook file and run it on [Google Colab](https://colab.research.google.com/) or run it on your system by setting up the Python environment.

# Before you start

## Create and Deploy Your Free Tier Operational cluster on Capella

To get started with Couchbase Capella, create an account and use it to deploy a forever free tier operational cluster. This account provides you with a environment where you can explore and learn about Capella with no time constraint.

To know more, please follow the [instructions](https://docs.couchbase.com/cloud/get-started/create-account.html).

### Couchbase Capella Configuration

When running Couchbase using [Capella](https://cloud.couchbase.com/sign-in), the following prerequisites need to be met.

* Create the [database credentials](https://docs.couchbase.com/cloud/clusters/manage-database-users.html) to access the travel-sample bucket (Read and Write) used in the application.
* [Allow access](https://docs.couchbase.com/cloud/clusters/allow-ip-address.html) to the Cluster from the IP on which the application is running.

# Install necessary libraries

In [ ]:
!pip install couchbase==4.3.6 transformers==4.52.2 sentence_transformers==4.1.0 langchain-community==0.3.24

# Imports

In [ ]:
from pathlib import Path
from datetime import timedelta
from transformers import pipeline, AutoModel, AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import (ClusterOptions, ClusterTimeoutOptions,
                               QueryOptions)
import couchbase.search as search
from couchbase.options import SearchOptions
from couchbase.vector_search import VectorQuery, VectorSearch
import uuid

# Prerequisites
In order to run this tutorial, you will need access to a Couchbase Cluster with Full Text Search service either through Couchbase Capella or by running it locally and have credentials to acces a collection on that cluster:

In [14]:
import getpass
couchbase_cluster_url = input("Couchbase Cluster URL:")
couchbase_username = input("Couchbase Username:")
couchbase_password = getpass.getpass("Couchbase password:")
couchbase_bucket = input("Couchbase Bucket:")
couchbase_scope = input("Couchbase Scope:")
couchbase_collection = input("Couchbase Collection:")

# Couchbase Connection
In this section, we first need to create a `PasswordAuthenticator` object that would hold our Couchbase credentials:

In [15]:
auth = PasswordAuthenticator(
    couchbase_username,
    couchbase_password
)

Then, we use this object to connect to Couchbase Cluster and select specified above bucket, scope and collection:

In [16]:
print("Connecting to cluster at URL: " + couchbase_cluster_url)
cluster = Cluster(couchbase_cluster_url, ClusterOptions(auth))
cluster.wait_until_ready(timedelta(seconds=5))

bucket = cluster.bucket(couchbase_bucket)
scope = bucket.scope(couchbase_scope)
collection = scope.collection(couchbase_collection)
print("Connected to the cluster")

Connecting to cluster at URL: couchbase://localhost
Connected to the cluster


# Creating Couchbase Vector Search Index
In order to store generated with Hugging Face embeddings onto a Couchbase Cluster, a vector search index needs to be created first. We included a sample index definition that will work with this tutorial in a file named `huggingface_index.json` located in the folder with this tutorial. The definition can be used to create a vector index using Couchbase server web console, on more information on vector indexes, please read [Create a Vector Search Index with the Server Web Console](https://docs.couchbase.com/server/current/vector-search/create-vector-search-index-ui.html). Please note that the index is configured for documents from bucket `hugginface`, scope `_default` and collection `huggingface` and you will have to edit `source` and document type name in the index definition file if your collection, scope or bucket names are different.

Here, our code verifies the existence of the index and will throw an exception if the index has not been found:

In [17]:
search_index_name = couchbase_bucket + "._default.vector_test"
search_index = cluster.search_indexes().get_index(search_index_name)
print("Found index: " + search_index_name)

Found index: pdf-docs._default.vector_test


# Hugging Face Initialization

In [25]:
embedding_model = HuggingFaceEmbeddings()
print("Initialized successfully")

/var/folders/zd/hmksn4rj5kd_pj7w7bw48y9m0000gn/T/ipykernel_33093/3783819977.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


Initialized successfully


# Embedding Documents
After initializing Hugging Face transformers library, it can be used to generate vector embeddings for user input or predefined set of phrases. Here, we're generating 2 embeddings for contained in the array strings:

In [26]:
texts = [
    "Couchbase Server is a multipurpose, distributed database that fuses the strengths of relational databases such as SQL and ACID transactions with JSON’s versatility, with a foundation that is extremely fast and scalable.",
    "It’s used across industries for things like user profiles, dynamic product catalogs, GenAI apps, vector search, high-speed caching, and much more.",
    input("Enter custom embedding text:")
]
embeddings = []
for i in range(0, len(texts)):
    embeddings.append(embedding_model.embed_query(texts[i]))

# Storing Embeddings in Couchbase
Generated embeddings are then stored as vector fields inside documents that can contain additional information about the vector, including the original text. The documents are then upserted onto the couchbase cluster:

In [27]:
for i in range(0, len(texts)):
    doc = {
        "id": str(uuid.uuid4()),
        "text": texts[i],
        "vector": embeddings[i],
    }
    collection.upsert(doc["id"], doc)

# Searching For Embeddings
After the documents are upserted onto the cluster, their vector fields will be added into previously imported vector index. Later, new embeddings can be added or used to perform a similarity search on the previously added documents:

In [29]:
def search_similar(text):
    print("Vector similarity search for phrase: \"" + text + "\"")
    search_embedding = embedding_model.embed_query(text)
    
    search_req = search.SearchRequest.create(search.MatchNoneQuery()).with_vector_search(
        VectorSearch.from_vector_query(
            VectorQuery(
                "vector", search_embedding, num_candidates=1
            )
        )
    )
    result = scope.search(
        "vector_test", 
        search_req, 
        SearchOptions(
            limit=13, 
            fields=["vector", "id", "text"]
        )
    )
    for row in result.rows():
        print("Found answer: " + row.id + "; score: " + str(row.score))
        doc = collection.get(row.id)
        print("Answer text: " + doc.value["text"])
        
search_similar("name a multipurpose database with distributed capability")
print("------")
search_similar(input("Enter custom search phrase:"))

Vector similarity search for phrase: "name a multipurpose database with distributed capability"
Found answer: 169df8dd-2aa4-4e43-889d-c8cdb0041927; score: 0.9951430384777361
Answer text: Get Started Couchbase Capella (DBaaS) Explore Couchbase Capella, our fully-managed database as a service offering. Take the complexity out of deploying, managing, scaling, and securing Couchbase in the public cloud. Store, query, and analyze any amount of data — and let us handle more of the administration — all in a few clicks.  Couchbase Capella    Capella Columnar (RT-OLAP) Capella Columnar is a real-time analytical database (RT-OLAP) for real time apps and operational intelligence. Capella Columnar is a standalone, cloud-only offering from Couchbase under the Capella family of products.  Capella Columnar  Couchbase Server Explore Couchbase Server, a modern, distributed document database with all the desired capabilities of a relational database and more. It exposes a scale-out, key-value store with